# CS 316 : Introduction to Deep Learning
## Lab 04 : Linear Regression
### Dr. Abdul Samad

# Instructions
1. Please rename your notebook as *Lab_4_aa1234.ipynb* before the final submission. Notebooks which do not follow appropriate naming convention will not be graded. 

2. Filling out the feedback form on Canvas is mandatory. Failure to do so will result in you losing 10% of your grade for this lab.

3. You have one week to submit this lab. Late submissions will be graded with a 30% penalty. No submission will be accepted after two weeks.

4. Please submit your own work. If you have any questions, please feel free to reach out to the course instructor , RA or TA.

# Setup

In [ ]:
# Importing the essentials
%matplotlib inline
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import os
import random
import urllib.request
import pandas as pd
np.random.seed(42)



# Task Overview
Create a linear regression model using the Boston Housing Dataset to find the set of equations to predict the price of a house. This lab has been divided into small subtasks to make it easier to follow.

## [10 Points] Task 01 - Download Boston Housing Dataset

A suitable dataset is required in order to obtain a linear regression model with reasonable accuracy. There are numerous datasets available, and many of them can be found online. You can also download the dataset manually, but scripting its retrieval is more convenient and may save time.
We'll use the urllib.request package to download the dataset.

 The following is an example of using 'urllib.request' to download a cat photo. 

In [ ]:
# Link from where to download.
cat_pic = 'http://i3.ytimg.com/vi/J---aiyznGQ/mqdefault.jpg'
# Path where you want to save.
save_path = os.path.join("Good_Luck_Charms", "cat.jpg")
# If folder doesn't exist create it.
if not os.path.isdir("Good_Luck_Charms"):
  os.mkdir("Good_Luck_Charms")
# If we have already save the file
if os.path.exists(save_path):
  print("File already exists")
else:
  try:
    urllib.request.urlretrieve(cat_pic, save_path)
    print(f"Cat picture added status: {os.path.exists(save_path)}")
  except:
    raise Exception(f"Something went wrong. Check your internet and download link.")

**Your task is to download the Boston Housing Dataset by completing the function `get_boston_housing_dataset`.**

**Link to Dataset**: [https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv](https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv)

In [ ]:
# TODO: Complete get_boston_housing_dataset
def get_boston_housing_dataset(filename):
    """
    Download the boston housing dataset. 
    If the file already exists, don't download the data again.
    Add appropriate error handling as well.

    Args:
        filename (string): The path where you want to save the file.
    Return:
        bool : True, if the file already exists, or if you successfully downloaded the data.
    """
    ...

if not os.path.isdir("datasets"):
  os.mkdir("datasets")
BostonHousingCSV = os.path.join("datasets", "BostonHousing.csv")
print(get_boston_housing_dataset(BostonHousingCSV))

## Exploratory Data Analysis

After downloading the csv file, let's observe the data and try to understand what it means. **The explanation for each column can be found at the end of the notebook in the section titled Explanations.**

In [ ]:
# Load the dataset
BostonHousingDataset = pd.read_csv(BostonHousingCSV)
# View the top rows of the dataset
BostonHousingDataset.head()

There is a lot of data. Most of the variables are unlikely to have a significant impact on the price. To avoid dealing with irrelevant columns, let us first determine which variables have a strong correlation with median price.

In [ ]:
display.set_matplotlib_formats('svg')
plt.figure(figsize=(8,8))
correlation_matrix = BostonHousingDataset.corr().round(2)
sns.heatmap(data=correlation_matrix, annot=True)
plt.show()

According to the heatmap, the two most important factors influencing price are **`lstat`** (Percentage of lower status of the population) and **`rm`** (Average number of rooms per dwelling). Consequently, we will be keep only these two columns. We can't add biases because we're only using matrices to find weights for each column. **As a result, we can add another column with a constant value of 1 that will serve as the bias.**

In [ ]:
# Extract only the relevant columns. X: Input, Y: Output
# iloc specifies the position of the column we want to select. :, part means all rows. ,-1 means last column
output = BostonHousingDataset.iloc[:,-1] # we can also do BostonHousingDataset['medv']
Y = np.array(output.values)
# X = np.array(BostonHousingDataset[['lstat','rm','ptratio']].values)
X = np.array(BostonHousingDataset[['lstat','rm']].values)

In [ ]:
ones = np.ones(shape=(len(X),1))
X_with_bias = np.concatenate((X,ones),axis=1)

## [20 Points] Task 02 - Train Test Split

To determine whether regression is effective, we can split our data into a training and test set and compare performance. Before splitting data, make sure to shuffle it because data is sometimes arranged in a specific order, such as all low-end houses first. If the data is not properly shuffled, it may fail to learn patterns. So, to avoid bias in your training data, shuffle it before splitting.








**Your task is to split the dataset into training and test dataset by completing the function `train_test_split(inputs,outputs,test_size,seed)`**

**Important note on seed**: Reproducability is extremely important. Sometimes, you might find a good seed for a split. When dealing with random shuffling, you will get different results everytime you run. You can set a global seed by using `np.random.seed(42)` however that sets the global seed. It is useful over here to not change the global seed here, since if you want to reproduce the same result, you will have to run the entire code again. The solution to this problem is that you create a generator with a specific seed and use that for generation. [You can find more about setting seeds here](https://towardsdatascience.com/stop-using-numpy-random-seed-581a9972805f)

In [ ]:
def train_test_split(inputs,outputs,test_size,seed = 0):
    """
    Splits the data into training and test sets.
    Return 4 numpy arrays. X_train, X_test, Y_train, Y_test
    where training data is test_size proportion of data provided.

    Args:
        inputs [np.array] : numpy array of input data
        outputs [np.array]: numpy array of output labels
        test_size [float]: proportion of data to be used as test data. e.g. 0.2 means 20% of data is used for test data.
        seed [int]: A seed to create random number generator. (For reproducability) 
    """
   
    rng = np.random.default_rng(seed)
    assert(len(inputs) == len(outputs))
    assert(test_size <= 1.0)
    assert(test_size >= 0.0)
    num_samples = len(inputs)
    num_train = int(num_samples * (1.0 - test_size))
    ...
SEED = 0
X_train, X_test, Y_train, Y_test = train_test_split(X_with_bias,Y,0.2,seed = SEED)


## [10 Points] Task 03 - Analytic Solution



**In this Task, you are required to compute the analytic solution by completing the function `analytical_solution`**

In [ ]:
#TODO: Complete analytical solution
def analytical_solution(X,y):
    """
    Returns the analytical solution to the dataset
    Args:
        X [np.array] : Inputs 
        y [np.array]: Outputs
    """
    ...


w = analytical_solution(X_with_bias,Y)
print(w)


## [20 Points] Task 04 - Evaluation

Mean Squared error is defined as

$$\text{MSE} = \dfrac{1}{n}\sum\limits_{i=1}^{n}(y_i-\hat{y}_i)^2$$

where $y_i$ is your actual value and $\hat{y}_i$ is the predicted value.

R2 score is also called the coefficient of determination. It basically is a statistical measure in a regression model that determines the proportion of variance in the dependent variable that can be explained by the independent variable. In simple words, how much of the output can actually be explained by the model or simply R2 just shows how well the data fit the regression model (the goodness of fit).

 $$R^2 = 1 - \dfrac{RSS}{TSS}$$

**RSS** is the sum of squares of residuals defined as
$$\sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

where $y_i$ is the actual value while $\hat{y_i}$ is what you predict.

**TSS** is defined as the Total sum of squares or simply the variance of our output variable. 

$$\sum_{i=1}^{n} (y_i - \bar{y})^2$$


where $\bar{y}$ is the mean value. 

Intuitively RSS/TSS is a ratio telling you how much a model sucks. So if RSS is 0, that means your model fits the data perfectly that is why 1 - RSS/TSS is **1. That's the best possible score you can get.** Furthemore, if your prediction is generally far from reality, your RSS would be pretty high, and that would make the score low. 

**In this task you are required to complete the functions `mean_squared_error` and `r2score`.**


In [ ]:
#TODO: Complete mean_squared_error
def mean_squared_error(y_actual,y_prediction):
    """
    Returns mean squared error using prediction and actual data
    Args:
        y_actual [np.array]     : Actual output vector 
        y_prediction [np.array] : Predicted output vector
    """
    ...

#TODO: Complete r2score
def r2score(y_test,y_pred):
    """
    Returns the R2 score for the given test outputs and predictions
    Args:
        y_test [np.array]: Test outputs
        y_pred [np.array]: Predictions
    """
    ...


## [40 Points] Task 05 - Gradient Descent



In this task, you will be implementing the Gradient Descent with and without the JAX libary. In both cases, you are required to train the model for 50 epochs and update the weights accordingly.

In [ ]:
# We would use X instead of X_with_bias since we have a b (bias) component)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,0.2,seed=SEED)
# Initialize weights
w = np.zeros(X_train.shape[0])
b = 0
lr = 0.005
loss_graph = []


### [20 Points] Part A : Without JAX

In [ ]:
#TODO: Write the main loop of Gradient Descent without using JAX
...
print(w,b)


#TODO: Report the MSE and R2 score
...
r2s = r2score(Y_test,y_pred)
mse = mean_squared_error(Y_test,y_pred)
print(f"R2 score after 50 epochs: {r2s}")
print(f"Mean Square Error after 50 epoch: {mse}")

### [20 Points] Part B: With JAX

In [ ]:
import jax
import jax.numpy as jnp

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,0.2,seed=SEED)
w_jax = jnp.zeros(X_train.shape[0])
b_jax = 0.
lr = 0.003

In [ ]:
#TODO: Write the main loop of Gradient Descent using JAX
...
print(w_jax,b_jax)

#TODO: Report the MSE and R2 score
...
r2s_jax = r2score(Y_test,y_pred)
mse_jax = mean_squared_error(Y_test,y_pred)
print(f"R2 score after 50 epochs: {r2s_jax}")
print(f"Mean Square Error after 50 epoch: {mse_jax}")




<h2 id="Boston"> Boston Housing Variables Meaning</h2>

`crim` : Per capita crime rate by town

`zn` : Proportion of residential land zoned for lots over 25,000 sq. ft

`indus` : Proportion of non-retail business acres per town

`chas` : Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)

`nox` : Nitric oxide concentration (parts per 10 million)

`rm` : Average number of rooms per dwelling

`age` : Proportion of owner-occupied units built prior to 1940

`dis` : Weighted distances to five Boston employment centers

`rad` : Index of accessibility to radial highways

`tax` : Full-value property tax rate per $10,000

`ptratio` : Pupil-teacher ratio by town

`b` : $1000(Bk — 0.63)^2$, where Bk is the proportion of [people of African American descent] by town

`lstat` : Percentage of lower status of the population

`medv` : Median value of owner-occupied homes in $1000s
## Mean Squared Error

$$\text{MSE} = \dfrac{1}{n}\sum\limits_{i=1}^{n}(y_i-\hat{y}_i)^2$$

It is simply on average how much your data deviates from the mean squared. (We square so the deviation doesn't get cancelled out).

## R2/R-Squared Score

R2 score is **how much** our input variables **determine** our output according to our model.

$$R^2 = 1 - \dfrac{RSS}{TSS}$$

__RSS__ : __sum of squares of residuals__  
$$\sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

$y_i$ is the actual value while $\hat{y_i}$ is what you predict.


__TSS__ : __Total sum of squares__ or __variance of our output variable__. 

$$\sum_{i=1}^{n} (y_i - \bar{y})^2$$


where $\bar{y}$ is the mean value. 


When __RSS__ IS 0, that means your model has the perfect weights to predict the price according to the data you are testing on. When that is the case value is $1$. 

In practice, __RSS__ may not be 0, since there is noise and you might be eliminating other variables. If your R2 score is 0 or less, then your model is not a usable fit. Prediction that the price is always mean will give us R2 score of 0. The better the fit, the better the model



